# Plot the top 35 players by total points since the 1992-1993 NBA Season.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import plotly.express as px


In [2]:
# Setup the transport and client
_transport = RequestsHTTPTransport(
    url='http://209.38.172.107/graphql/',
    use_json=True,
)

client = Client(
    transport = _transport,
    fetch_schema_from_transport=True,
)

# Define Query
query = gql('''
query playerPointTotals {
  playerTotalsAll(limit: 35, ordering: "-points") {
    playerName
    position
    team
    season
    minutesPg
    points
    id
  }
}
''')

# Execute and store response

response = client.execute(query)

print(response)

{'playerTotalsAll': [{'playerName': 'Kobe Bryant*', 'position': 'SG', 'team': 'LAL', 'season': 2006, 'minutesPg': '3277.0', 'points': 2832, 'id': '10755'}, {'playerName': 'James Harden', 'position': 'PG', 'team': 'HOU', 'season': 2019, 'minutesPg': '2867.0', 'points': 2818, 'id': '3118'}, {'playerName': 'Kevin Durant', 'position': 'SF', 'team': 'OKC', 'season': 2014, 'minutesPg': '3122.0', 'points': 2593, 'id': '6229'}, {'playerName': 'Russell Westbrook', 'position': 'PG', 'team': 'OKC', 'season': 2017, 'minutesPg': '2802.0', 'points': 2558, 'id': '4778'}, {'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1993, 'minutesPg': '3067.0', 'points': 2541, 'id': '17728'}, {'playerName': 'Michael Jordan*', 'position': 'SG', 'team': 'CHI', 'season': 1996, 'minutesPg': '3090.0', 'points': 2491, 'id': '16287'}, {'playerName': 'LeBron James', 'position': 'SF', 'team': 'CLE', 'season': 2006, 'minutesPg': '3361.0', 'points': 2478, 'id': '10931'}, {'playerName': 'Kevin Dura

### Create data frame from response. Ensure minutes are floating points, data types are correct.

In [3]:
# Convert JSON to DF

df = pd.DataFrame(response['playerTotalsAll'])

df['minutesPg'] = df['minutesPg'].astype(float)
df['points'] = df['points'].astype(int)

# Sort data by points

df_sorted = df.sort_values('points', ascending=False).head(35)

df_sorted.head()

,playerName,position,team,season,minutesPg,points,id
0,Kobe Bryant*,SG,LAL,2006,3277.0,2832,10755
1,James Harden,PG,HOU,2019,2867.0,2818,3118
2,Kevin Durant,SF,OKC,2014,3122.0,2593,6229
3,Russell Westbrook,PG,OKC,2017,2802.0,2558,4778
4,Michael Jordan*,SG,CHI,1993,3067.0,2541,17728


### Create player_season column to represent y-axis data containing individual Player Name (Season) for each row.

In [4]:
df_sorted['player_season'] = df_sorted.apply(lambda x: f"{x['playerName']} ({x['season']})", axis=1)

In [5]:
# Catch and remove stupid warning

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    fig = px.bar(df_sorted, y='player_season', x='points', color='position', hover_data=['season', 'team', 'minutesPg'], labels={'points': 'Total Points', 'playerName': 'Player Name', 'position': 'Position', 'player_season': 'Player (Season)'}, title='Top 35 NBA Players by Points since the 1992-1993 season.')
    
    
    fig.layout.autosize = False
    fig.layout.height = 1000
    fig.layout.width = 1600

    fig.show()

